<a href="https://colab.research.google.com/github/MarlonGrandy/LSTMMusicGeneration/blob/main/LSTMMusicGenerationPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import statements
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from music21 import converter, instrument, note, chord, stream, duration
import glob
import os
from itertools import chain
import copy
import numpy as np
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from keras.layers import CuDNNLSTM,Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from numpy import argmax
from fractions import Fraction
import gc
import random
import keras.utils

In [3]:
def data_extractor(directory):
  """
    Summary:
    Function converts midi files to metadata and appends nested metadata lists into one large list
    composed of all the songs in the dataset.

    Parameters:
    directory: String representation of directory where midi files are located

    Returns:
    list: sequential midi file metadata 
   """
  notes = []
  offsets = []
  durations = []
  for file in glob.glob(directory):
          mid = converter.parse(file)
          notes_to_parse = None
          prev_offset = 0
          
          try: 
              s2 = instrument.partitionByInstrument(mid)
              notes_to_parse = s2.parts[0].recurse() 
              
          except: 
              notes_to_parse = mid.flat.notes

          for i,element in enumerate(notes_to_parse):
              if isinstance(element, note.Note):
                  notes.append(str(element.pitch))
                  
                  durations.append(str(element.quarterLength))
                  
                  offset_dif = float(element.offset-prev_offset)
              
                  offsets.append(round(offset_dif,3))
                  prev_offset = element.offset
             
                 
              elif isinstance(element, chord.Chord):
                  notes.append('.'.join(str(n) for n in element.normalOrder))
                  offset_dif = float(element.offset-prev_offset)

                  durations.append(str(element.quarterLength))
                  
                  offsets.append(round(offset_dif,3))
                  prev_offset = element.offset

  return [notes,offsets,durations]

In [4]:
data = data_extractor("/content/drive/MyDrive/midi_stuff/*.mid")
note_data = data[0]
offset_data = data[1]
duration_data = data[2]

unique_note_number = len(list(set(note_data)))
unique_notes = sorted(list(set(note_data)))
unique_offset_number = len(list(set(offset_data)))
unique_offsets = sorted(list(set(offset_data)))
unique_duration_number = len(list(set(duration_data)))
unique_durations = sorted(list(set(duration_data)))

In [5]:
def one_hot_encode(vector, all_values):
  encoded_vectors = []
  int_to_index = dict((c, i) for i, c in enumerate(all_values))
  for i in vector:
    zero = [0]*(len(all_values)-1)
    zero.insert(int_to_index[i],1)
    encoded_vectors.append(zero)
  
  return encoded_vectors

In [6]:
from numpy import argmax
def one_hot_decode(vector,all_values):
  decoded_vector = []
  index_to_int = dict((i, c) for i, c in enumerate(all_values))
  decoded_vector.append(index_to_int[argmax(vector)])
  return decoded_vector

In [7]:
segment_length = 64
def make_segments(data_array,unique_values, seq_length = segment_length):
  input_seq = []
  output_seq = []

  processed_data = one_hot_encode(data_array,unique_values)
  
  for i in range(0,len(processed_data) - seq_length,1):
    input_seq.append([processed_data[i:i+seq_length]])
    output_seq.append(processed_data[seq_length + i])

  del processed_data;gc.collect()
  
  input_seq = np.reshape(input_seq,(len(input_seq), segment_length,len(unique_values)))
  output_seq = np.array(output_seq)

  return input_seq, output_seq

In [8]:
note_model_data = make_segments(data_array = note_data, unique_values = unique_notes)
offset_model_data = make_segments(data_array = offset_data, unique_values = unique_offsets)
duration_model_data = make_segments(data_array = duration_data, unique_values = unique_durations)

In [9]:
X_train_note, X_test_note, y_train_note, y_test_note = train_test_split(note_model_data[0], note_model_data[1],test_size=0.2)
X_train_off, X_test_off, y_train_off, y_test_off = train_test_split(offset_model_data[0], offset_model_data[1],test_size=0.2)
X_train_dur, X_test_dur, y_train_dur, y_test_dur = train_test_split(duration_model_data[0], duration_model_data[1],test_size=0.2)

In [10]:
del note_model_data
del offset_model_data
del duration_model_data
gc.collect()

0

In [11]:
def make_model(input_shape,output_shape):
  model = Sequential()
  model.add(CuDNNLSTM(512,input_shape=input_shape,return_sequences=False))
  model.add(Dropout(0.25))
  model.add(Dense(128))
  model.add(Dense(output_shape,activation = 'softmax'))

  model.compile(
      loss='categorical_crossentropy',
      optimizer=Adam(learning_rate = .001),
      metrics=['accuracy']
  )
  return model

In [12]:
model_notes = make_model(input_shape = (segment_length,unique_note_number), output_shape = unique_note_number)
model_offsets = make_model(input_shape = (segment_length,unique_offset_number), output_shape = unique_offset_number)
model_durations = make_model(input_shape = (segment_length,unique_duration_number), output_shape = unique_duration_number)

In [13]:
my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]

In [ ]:
history_note = model_notes.fit(X_train_note,y_train_note,epochs=150, validation_split = 0.2, callbacks = my_callbacks)

Epoch 1/150
1387/1387 [==============================] - 28s 16ms/step - loss: 4.1239 - accuracy: 0.0706 - val_loss: 3.7334 - val_accuracy: 0.1064
Epoch 2/150
1387/1387 [==============================] - 20s 15ms/step - loss: 3.5292 - accuracy: 0.1414 - val_loss: 3.3729 - val_accuracy: 0.1676
Epoch 3/150
1387/1387 [==============================] - 20s 15ms/step - loss: 3.2804 - accuracy: 0.1906 - val_loss: 3.1766 - val_accuracy: 0.2114
Epoch 4/150
1387/1387 [==============================] - 20s 14ms/step - loss: 3.0365 - accuracy: 0.2330 - val_loss: 3.0309 - val_accuracy: 0.2416
Epoch 5/150
1387/1387 [==============================] - 20s 14ms/step - loss: 2.8655 - accuracy: 0.2679 - val_loss: 2.9091 - val_accuracy: 0.2734
Epoch 6/150
1387/1387 [==============================] - 20s 14ms/step - loss: 2.6746 - accuracy: 0.3084 - val_loss: 2.7950 - val_accuracy: 0.3004
Epoch 7/150
1387/1387 [==============================] - 20s 14ms/step - loss: 2.4595 - accuracy: 0.3558 - val_loss: 2

In [ ]:
history_offset = model_offsets.fit(X_train_off,y_train_off,epochs=150, validation_split = 0.2,callbacks = my_callbacks)

In [ ]:
history_duration = model_durations.fit(X_train_dur,y_train_dur,epochs=150, validation_split = 0.2,callbacks = my_callbacks)

In [ ]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  
    preds = preds / np.sum(preds)                
    probas = np.random.multinomial(1, preds, 1)  
    return np.argmax(probas)                     

In [ ]:
def music_maker(seed_vec, model_type, number_unique, unique, diversity, num_notes = 64):
  music = []
  arr = np.zeros((len(seed_vec)+num_notes,number_unique))
  for c,i in enumerate(seed_vec):
    arr[c] = i

  for i in range(0,num_notes,1):
    d_arr = np.zeros(number_unique)
    pred = model_type.predict(np.reshape(arr[i:len(arr)-num_notes+i], (1, len(arr[i:len(arr)-num_notes+i]),number_unique)),verbose = 0)[0]
    diverse = sample(pred, diversity)
    d_arr[diverse] = 1
    music.append(one_hot_decode(d_arr,all_values = unique)[0])

    arr[len(seed_vec)+i] = d_arr
  
  return music

In [ ]:
randnum = random.randrange(0,len(X_test_note))
test_note = X_test_note[randnum]
test_offset = X_test_off[randnum]
test_dur = X_test_dur[randnum]
generated_music_note = music_maker(test_note, model_notes, unique_note_number, unique_notes, 0.7)
generated_music_offset = music_maker(test_offset,model_offsets,unique_offset_number, unique_offsets, 0.7)
generated_music_duration = music_maker(test_dur, model_durations, unique_duration_number, unique_durations, 0.2)


In [ ]:
seed_notes = []
seed_offsets = []
seed_durations = []
for n in test_note:
  seed_notes.append(one_hot_decode(n ,unique_notes)[0]) 
for o in test_offset:
  seed_offsets.append(one_hot_decode(o ,unique_offsets)[0]) 
for d in test_dur:
  seed_durations.append(one_hot_decode(d, unique_durations)[0])

In [ ]:
def to_midi(notes, offsets, durations):
  """
  Summary:
  Takes midi metadata and converts it to a Music21 stream. The stream can then easily be converted into a midi file.

  Parameters:
  metadata: midi metadata (notes,chords,offsets)

  Returns:
  list: Music21 stream object
  """
  offset = offsets[0]
  s = stream.Stream()
  for i,ele in enumerate(notes):
    if ele[0].isalpha():
      n = note.Note(ele)
      try:
        n.quarterLength = float(durations[i])
      except:
        n.quarterLength = Fraction(durations[i])

      s.insert(offset,n)

      offset += offsets[i]
    else:
      chords = list(map(int,ele.split('.')))
      c = chord.Chord(chords)

      try:
        c.quarterLength = float(durations[i])
      except:
        c.quarterLength = Fraction(durations[i])

      s.insert(offset,c)

      offset += offsets[i]
  return s

In [ ]:
to_midi(generated_music_note,generated_music_offset, generated_music_duration).write('midi', "generated_classical.mid")
to_midi(seed_notes,seed_offsets,seed_durations).write('midi', "seed_classical.mid")
